# CmCt Histogram Tool

This Jupyter notebook can be used to plot the comparison results from the Cryosphere model Comparison tool (CmCt, https://ggsghpcc.sgt-inc.com/cmct/) directly in this notebook and/or saving the plot on the user's local machine.

The Cryosphere model Comparison tool (CmCt) is a web-based ice sheet model validation tool that has been developed by NASA to facilitate direct comparison between satellite observational data and various ice sheet models. The CmCt allows the user to take advantage of several decades worth of satellite-based observations from Greenland and Antarctica. The CmCt is used to compare ice sheet models provided by the user with remotely sensed satellite ICESat (Ice, Cloud, and land Elevation Satellite) laser altimetry, ERS-1, ERS-2, and Envisat radar altimetry data, and GRACE (Gravity Recovery and Climate Experiment) satellite data. For the comparison with GRACE data, the uploaded ice sheet model can also be compared with different solutions: GRACE Mascon or Spherical Harmonics. 

The user can upload the histogram text file that was created by the CmCt. After upload these model files will be in a folder located within the user's home data directory. The user can plot the data using the plot button which appears after a file has been uploaded. The script will create a cummulative distribution function (CDF) and a probablility density function (PDF) side by side for quick assessment of the comparison results.

After running the plotting cells, the histograms can be viewed within the notebook. The user also has an option to save the plot. The plot will be saved in the user's home data directory on the gateway. The user can also choose to download the result by clicking on the download botton. The plot will appear in a new window and by right clicking the image it can be downloaded using several different options.

### To run the tool:

#### Select Kernel: Restart & Run All from the top menu.
#### Once all cells have refreshed, click the Hide Code Cells toggle button for easier navigation.

In [1]:
#Importing Python Libraries
import hublib.use
%use anaconda-6

import os,sys
# print ('sys.executable: ', sys.executable)

# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob as glob
from matplotlib import rc
rc('mathtext', default='regular')

sys.path.insert(0, os.path.abspath('..'))
from hublib.ui import FileUpload, Download

import ipywidgets as widgets
from IPython.display import display,clear_output

import gc

import requests
import atexit

<IPython.core.display.Javascript object>

In [2]:
runButton = hublib.ui.RunAllButton(style='info')
display(runButton)

Button(button_style='info', description='Run All Cells', style=ButtonStyle())

In [3]:
hideCodeButton = hublib.ui.HideCodeButton(style='success')
display(hideCodeButton)

Button(button_style='success', description='Hide Code Cells', style=ButtonStyle())

In [4]:
# create histogram plot from input .txt file
def create_histogram(file_name):
    output_widget.clear_output()
    update_progress(0)
    
    plt.close('all')
    #set fontsize
    matplotlib.rcParams.update({'font.size': 10})

    #set up variable names
    varnames   = ('bin_low', 'bin_high','nbin','PDF','CDF')

    #set up variable formats
    varformats = ('f4', 'f4','i2','f8','f8')

    dtype = {'names': varnames, 'formats': varformats}

    update_progress(0.1)

    fbase = os.path.basename(file_name)

    #use built in numpy function to read in data, and ignore the header
    #this will return a numpy array of tuples
    try:
        data  = np.loadtxt(file_name,comments="#",dtype=dtype)
    except:
        output_widget.clear_output()
        with output_widget:
            print('ERROR: Program could not decode file. May be incorrect file type.')
        return 'stopped'

    #to separate the data into separate numpy arrays
    try:
        bin_low    = data["bin_low"]
        bin_high   = data["bin_high"]
        nbin       = data["nbin"]
        PDF        = data["PDF"]
        CDF        = data["CDF"]
    except:
        with output_widget:
            print('ERROR: file is missing crucial data. May be incorrect file type.')
        return 'stopped'
    
    update_progress(0.2)
    
    numrecs = len(nbin)

    plt.ion()
    
    fig = plt.figure(figsize=(16,10))

    #----------------------------------------------------------------------
    #First plot will be histogram with CDF
    #----------------------------------------------------------------------
    update_progress(0.3)
    
    ax = plt.subplot(121)

    width = bin_high - bin_low
    p1 = plt.bar(bin_low,nbin,width,color="blue")
    
    plt.ylabel('Counts',rotation=90)
    plt.xlabel('Bins (m)')
    
    ax.set_ylim(ymin=0)
    xlabels1 = ax.get_xticklabels()
    ylabels1 = ax.get_yticklabels()

    plt.setp(xlabels1, rotation=90)

    update_progress(0.4)
    
    #plot the CDF on the RHS axis (it runs from 0-1)
    ax2 = plt.twinx()
    ax2.plot(bin_low,CDF,color="red",label='CDF')
    ax2.set_ylim(ymin=0)

    update_progress(0.5)
    
    #plot the y title 90 degrees.
    plt.ylabel('Probablility',rotation=90)

    plt.legend()
    plt.title('Cumulative Distribution Function (CDF)',fontsize=15)
    
    #----------------------------------------------------------------------
    #Second plot will be histogram with PDF
    #----------------------------------------------------------------------
    update_progress(0.6)
    
    ax3 = plt.subplot(122)

    p2 = plt.bar(bin_low,nbin,width,color="blue")
    plt.ylabel('Counts',rotation=90)
    plt.xlabel('Bins (m)')

    ax3.set_ylim(ymin=0)
    xlabels2 = ax3.get_xticklabels()
    ylabels2 = ax3.get_yticklabels()
    plt.setp(xlabels2, rotation=90)

    update_progress(0.7)
    
    #plot the PDF on the RHS axis 
    ax4 = plt.twinx()
    ax4.plot(bin_low,PDF,color="red",label='PDF')
    ax4.set_ylim(ymin=0)

    update_progress(0.8)
    
    #plot the y title 90 degrees.
    plt.ylabel('Probability Density',rotation=90)

    plt.legend()
    plt.title('Probability Density Function (PDF)',fontsize=15)

    plt.subplots_adjust(wspace=0.5)
    
    axes = plt.gca()
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    plt.suptitle('CmCt Comparison Histogram Plots', fontsize=25)
    plt.subplots_adjust(top=0.83)

    plt.savefig("CmCt_Histogram.png",dpi=100)
    
    update_progress(0.9)
    output_widget.clear_output(wait = True)
    with output_widget:
        plt.show()

In [5]:
#def default_cb(b):
#   example_plot.disabled = True
#   create_histogram(file_name)

# create histogram using uploaded data
def uploaded_cb(b):
    uploaded_data.disabled = True
    f.visible = False
    download_button(False)
    status = create_histogram(file_name)
    if status!='stopped':
        download_button(True)
    else:
        uploaded_data.disabled = True
    f.visible = True
    #example_plot.disabled = False
    
# Called when all files finish uploading
def done_cb(w, name):
    output_widget.clear_output()
    with output_widget:
        print("%s uploaded" % name[0])
    global file_name
    file_name = name[0]
    uploaded_data.layout.visibility = 'visible'
    uploaded_data.disabled = False
    # reset clears and re-enables the widget for more uploads
    w.reset()

# download histogram output plot    
def download_button(active):
    if active:
        d1 = Download(filename='CmCt_Histogram.png',label='Download Plot', style='success', icon='fa-arrow-circle-down')
        with button_output_widget:
            display(d1)
    else:
        button_output_widget.clear_output()
    
# update upload or plotting progress bar
def update_progress(progress):
    title = 'Plotting Data'
    bar_length = 20
    block = int(20.0*progress)
    text = title+" [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    output_widget.clear_output(wait = True)
    with output_widget:
        print(text)    

In [7]:
#CMCT File Upload
#run the cell below and upload a file
# todo: standardize input file paths

#example_data = widgets.Button(
#    description = 'Use Example Data',
#    button_style = 'info')
#example_data.on_click(default_cb)

# Plot button for uploaded data
uploaded_data = widgets.Button(
    description = 'Plot Data',
    button_style = 'info',
    layout = widgets.Layout(visibility = 'hidden'))
uploaded_data.on_click(uploaded_cb)

output_widget = widgets.Output()
button_output_widget = widgets.Output()

#example_plot = widgets.Button(description = 'Example Plot',button_style = 'info')
#example_plot.on_click(default_cb)

# Upload button for input data
global upload_directory
upload_directory = 'data'
f = FileUpload('',
    'Please Upload Your File',
    dir=upload_directory,
    cb=done_cb,
    maxsize='50M',
    width = '350px')
#dir(f)

In [8]:
# prepare example plot
file_name = 'CMCT_Example.histogram.txt'
create_histogram(file_name)

In [9]:
# display plot
display(output_widget)

Output()

In [10]:
# Enable plot download
download_button(True)
display(button_output_widget)

Output()

In [11]:
# Upload your own data

In [12]:
# Display file upload widget
display(f)
# informational:
#dir(f)

# Display plotting button for uploaded data
display(uploaded_data)

Button(button_style='info', description='Plot Data', layout=Layout(visibility='hidden'), style=ButtonStyle())